# setup

In [1]:
# import pdb

In [2]:
import pickle
import numpy as np

import torch
import matplotlib.pyplot as plt

from torch import nn, Tensor
# from jaxtyping import Float, Int
from typing import Optional, Callable, Union, List, Tuple

# from datasets import load_dataset

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# load labels

In [157]:
import json
with open('gemma-2-2b-20-gemmascope-res-16k-explanations.json', 'rb') as f:
    feat_labels_allData = json.load(f)

In [158]:
feat_labels_lst = [0 for i in range(feat_labels_allData['explanationsCount'])]
feat_labels_dict = {}
for f_dict in feat_labels_allData['explanations']:
    feat_labels_lst[int(f_dict['index'])] = f_dict['description']
    feat_labels_dict[int(f_dict['index'])] = f_dict['description']
    if int(f_dict['index']) == 0:
        print(f_dict['description'])

mentions of the letter 'Z' and variations of 'Z' in different contexts


In [159]:
len(feat_labels_dict)

16383

## search for features

In [ ]:
def find_indices_with_keyword(f_dict, keyword):
    """
    Find all indices of fList which contain the keyword in the string at those indices.

    Args:
    fList (list of str): List of strings to search within.
    keyword (str): Keyword to search for within the strings of fList.

    Returns:
    list of int: List of indices where the keyword is found within the strings of fList.
    """
    filt_dict = {}
    for index, string in f_dict.items():
        # split_list = string.split(',')
        # no_space_list = [i.replace(' ', '').lower() for i in split_list]
        # if keyword in no_space_list:
        if keyword in string:
            filt_dict[index] = string
    return filt_dict

In [ ]:
keyword = "number"
number_feats = find_indices_with_keyword(feat_labels_dict, keyword)

In [ ]:
keyword = "month"
month_feats = find_indices_with_keyword(feat_labels_dict, keyword)

In [ ]:
set(number_feats).intersection(month_feats)

{5769, 12790, 20260, 21029}

In [ ]:
number_feats[5769]

'numbers with possible time expressions like years, months, and hours'

In [ ]:
for common_feat in set(number_feats).intersection(month_feats):
    del number_feats[common_feat]

In [ ]:
set(number_feats).infor sampID, samp_common_feats in enumerate(common_feats_per_row):
    print('samp: ', sampID)
    for f_ind in samp_common_feats:
        print(f_ind, feat_labels_lst[f_ind])tersection(month_feats)

set()

# load model

In [8]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import numpy as np
import torch

In [9]:
from huggingface_hub import hf_hub_download, notebook_login
notebook_login()

In [10]:
torch.set_grad_enabled(False) # avoid blowing up mem

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",
    device_map='auto',
)

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [12]:
tokenizer =  AutoTokenizer.from_pretrained("google/gemma-2-2b")

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# load transformerlens

In [24]:
%%capture
!pip install transformer_lens

In [25]:
%%capture
from transformer_lens import HookedTransformer

# uses a lot of memory, use A100
model_2 = HookedTransformer.from_pretrained(
    "gemma-2-2b"
)

In [28]:
from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

# test prompts

In [ ]:
prompt = "thirteen fourteen fifteen sixteen "
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

1


# load sae

In [13]:
path_to_params = hf_hub_download(
    repo_id="google/gemma-scope-2b-pt-res",
    filename="layer_20/width_16k/average_l0_71/params.npz",
    force_download=False,
)

params.npz:   0%|          | 0.00/302M [00:00<?, ?B/s]

In [14]:
params = np.load(path_to_params)
pt_params = {k: torch.from_numpy(v).cuda() for k, v in params.items()}

In [15]:
import torch.nn as nn
class JumpReLUSAE(nn.Module):
  def __init__(self, d_model, d_sae):
    # Note that we initialise these to zeros because we're loading in pre-trained weights.
    # If you want to train your own SAEs then we recommend using blah
    super().__init__()
    self.W_enc = nn.Parameter(torch.zeros(d_model, d_sae))
    self.W_dec = nn.Parameter(torch.zeros(d_sae, d_model))
    self.threshold = nn.Parameter(torch.zeros(d_sae))
    self.b_enc = nn.Parameter(torch.zeros(d_sae))
    self.b_dec = nn.Parameter(torch.zeros(d_model))

  def encode(self, input_acts):
    pre_acts = input_acts @ self.W_enc + self.b_enc
    mask = (pre_acts > self.threshold)
    acts = mask * torch.nn.functional.relu(pre_acts)
    return acts

  def decode(self, acts):
    return acts @ self.W_dec + self.b_dec

  def forward(self, acts):
    acts = self.encode(acts)
    recon = self.decode(acts)
    return recon


In [16]:
sae = JumpReLUSAE(params['W_enc'].shape[0], params['W_enc'].shape[1])
sae.load_state_dict(pt_params)

<All keys matched successfully>

In [17]:
sae.cuda()

JumpReLUSAE()

## load sae weights


In [ ]:
weight_matrix = sae.W_dec.detach().cpu().numpy()
weight_matrix.shape

## get actv fns

In [18]:
prompt = "January February March April"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")

In [19]:
def gather_residual_activations(model, target_layer, inputs):
  target_act = None
  def gather_target_act_hook(mod, inputs, outputs):
    nonlocal target_act # make sure we can modify the target_act from the outer scope
    target_act = outputs[0]
    return outputs
  handle = model.model.layers[target_layer].register_forward_hook(gather_target_act_hook)
  _ = model.forward(inputs)
  handle.remove()
  return target_act

In [20]:
target_act = gather_residual_activations(model, 20, inputs)

Now, we can run our SAE on the saved activations.

In [21]:
sae_acts = sae.encode(target_act.to(torch.float32))
# recon = sae.decode(sae_acts)

In [22]:
sae_acts.shape

torch.Size([1, 5, 16384])

# steering fns

Steering sae fns

In [ ]:
def steer_by_sae_actvs(prompt, steer_vec, multp):
    tokens = model_2.to_tokens(prompt).to(device)
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
    target_act = gather_residual_activations(model, 20, inputs)
    sae_acts_3 = sae.encode(target_act.to(torch.float32))

    sae_acts_3  += multp * steer_vec # in-place
    recon = sae.decode(sae_acts_3)
    recon.shape

    # replace LLM actvs in that layer with decoder output

    layer_name = 'blocks.20.hook_resid_post'

    def patch_layer(
        orig_actvs: Float[Tensor, "batch pos d_model"],
        hook: HookPoint,
        LLM_patch: Float[Tensor, "batch pos d_model"],
        layer_to_patch: int,
    ) -> Float[Tensor, "batch pos d_model"]:
        if layer_to_patch == hook.layer():
            orig_actvs[:, :, :] = LLM_patch
        return orig_actvs

    hook_fn = partial(
            patch_layer,
            LLM_patch= recon,
            layer_to_patch = 20
        )

    # if you use run_with_cache, you need to add_hook before
    # if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
    # no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

    # rerun clean inputs on ablated model
    ablated_logits = model_2.run_with_hooks(tokens,
                        fwd_hooks=[
                            (layer_name, hook_fn),
                        ]
                    )

    next_token = ablated_logits[0, -1].argmax(dim=-1)
    next_char = model_2.to_string(next_token)
    print(next_char)

In [ ]:
def steer_by_sae_lastTok(prompt, steer_vec, multp):
    tokens = model_2.to_tokens(prompt).to(device)
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
    target_act = gather_residual_activations(model, 20, inputs)
    sae_acts_3 = sae.encode(target_act.to(torch.float32))

    sae_acts_3[:, -1, :]  += multp * steer_vec[:, -1, :] # in-place
    recon = sae.decode(sae_acts_3)
    recon.shape

    # replace LLM actvs in that layer with decoder output

    layer_name = 'blocks.20.hook_resid_post'

    def patch_layer(
        orig_actvs: Float[Tensor, "batch pos d_model"],
        hook: HookPoint,
        LLM_patch: Float[Tensor, "batch pos d_model"],
        layer_to_patch: int,
    ) -> Float[Tensor, "batch pos d_model"]:
        if layer_to_patch == hook.layer():
            orig_actvs[:, :, :] = LLM_patch
        return orig_actvs

    hook_fn = partial(
            patch_layer,
            LLM_patch= recon,
            layer_to_patch = 20
        )

    # if you use run_with_cache, you need to add_hook before
    # if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
    # no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

    # rerun clean inputs on ablated model
    ablated_logits = model_2.run_with_hooks(tokens,
                        fwd_hooks=[
                            (layer_name, hook_fn),
                        ]
                    )

    next_token = ablated_logits[0, -1].argmax(dim=-1)
    next_char = model_2.to_string(next_token)
    print(next_char)

LLM steering fns

In [54]:
def steer_by_LLM_lastTok(prompt, steer_vec, multp):
    tokens = model_2.to_tokens(prompt).to(device)
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
    target_act = gather_residual_activations(model, 20, inputs)

    target_act[:, -1, :]  += multp * steer_vec[:, -1, :] # in-place

    # replace LLM actvs in that layer with decoder output

    layer_name = 'blocks.20.hook_resid_post'

    def patch_layer(
        orig_actvs: Float[Tensor, "batch pos d_model"],
        hook: HookPoint,
        LLM_patch: Float[Tensor, "batch pos d_model"],
        layer_to_patch: int,
    ) -> Float[Tensor, "batch pos d_model"]:
        if layer_to_patch == hook.layer():
            orig_actvs[:, :, :] = LLM_patch
        return orig_actvs

    hook_fn = partial(
            patch_layer,
            LLM_patch= target_act,
            layer_to_patch = 20
        )

    # if you use run_with_cache, you need to add_hook before
    # if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
    # no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

    # rerun clean inputs on ablated model
    ablated_logits = model_2.run_with_hooks(tokens,
                        fwd_hooks=[
                            (layer_name, hook_fn),
                        ]
                    )

    next_token = ablated_logits[0, -1].argmax(dim=-1)
    next_char = model_2.to_string(next_token)
    print(next_char)

In [ ]:
def steer_by_LLM_decompSAE(prompt, patch, multp):
    # replace LLM actvs in that layer with decoder output

    layer_name = 'blocks.20.hook_resid_post'

    def patch_layer(
        orig_actvs: Float[Tensor, "batch pos d_model"],
        hook: HookPoint,
        LLM_patch: Float[Tensor, "batch pos d_model"],
        layer_to_patch: int,
    ) -> Float[Tensor, "batch pos d_model"]:
        if layer_to_patch == hook.layer():
            orig_actvs[:, :, :] = LLM_patch
        return orig_actvs

    hook_fn = partial(
            patch_layer,
            LLM_patch= patch,
            layer_to_patch = 20
        )

    # if you use run_with_cache, you need to add_hook before
    # if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
    # no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

    # rerun clean inputs on ablated model
    ablated_logits = model_2.run_with_hooks(tokens,
                        fwd_hooks=[
                            (layer_name, hook_fn),
                        ]
                    )

    next_token = ablated_logits[0, -1].argmax(dim=-1)
    next_char = model_2.to_string(next_token)
    print(next_char)

# steer by SAE using members not seqs

## get sv without subtraction- numerals

In [ ]:
words = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
numword_samps = []
for i in range(0, 10):
    samp = f"{words[i]}"
    numword_samps.append(samp)
inputs = tokenizer(numword_samps, return_tensors="pt", padding=True, truncation=True, max_length=300)['input_ids'].to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [ ]:
# nw_sv = sae_acts_1
mean_num_sv = sae_acts_1.mean(dim=0)
mean_num_sv = mean_num_sv.unsqueeze(0)
mean_num_sv.shape

torch.Size([1, 3, 16384])

### steer

In [ ]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    numword_samps.append(samp)

In [ ]:
steer_vec = mean_num_sv
multp = 1

for prompt in numword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

one two three four
5
two three four five
6
three four five six
7
four five six seven
8
five six seven eight
9
six seven eight nine
0
seven eight nine ten


eight nine ten eleven
2
nine ten eleven twelve
3


In [ ]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]}" # {words[i+3]}"
    numword_samps.append(samp)

In [ ]:
steer_vec = mean_num_sv
multp = 1

for prompt in numword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

one two three
4
two three four
5
three four five
6
four five six
7
five six seven
8
six seven eight
9
seven eight nine
0
eight nine ten
0
nine ten eleven
2


In [ ]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]}" # {words[i+2]} {words[i+3]}"
    numword_samps.append(samp)

In [ ]:
steer_vec = mean_num_sv
multp = 1

for prompt in numword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

one two
0
two three
0
three four
0
four five
6
five six
7
six seven
0
seven eight
0
eight nine
0
nine ten
0


In [ ]:
words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
months_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    months_samps.append(samp)

In [ ]:
steer_vec = mean_num_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

January February March April
5
February March April May
6
March April May June
7
April May June July
8
May June July August
9
June July August September


July August September October


August September October November


September October November December




In [ ]:
words = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
months_samps = []
for i in range(0, 5):
    samp = f"{words[i]} {words[i+1]} {words[i+2]}" # {words[i+3]}"
    months_samps.append(samp)

In [ ]:
# assume monday is the first day, aim to get 4 if ends on Wedn
steer_vec = mean_num_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
4
Tuesday Wednesday Thursday
5
Wednesday Thursday Friday


Thursday Friday Saturday


Friday Saturday Sunday




In [ ]:
# assume monday is the first day, aim to get 4 if ends on Wedn
steer_vec = mean_num_sv
multp = 3

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
4
Tuesday Wednesday Thursday
5
Wednesday Thursday Friday
6
Thursday Friday Saturday
0
Friday Saturday Sunday
0


In [82]:
words = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
tomorrowIs_samps = []
for i in range(0, 6):
    samp = f"Today is {words[i]}, tomorrow is"
    tomorrowIs_samps.append(samp)

In [83]:
# assume monday is the first day, aim to get 4 if ends on Wedn
steer_vec = mean_num_sv
multp = 1

for prompt in tomorrowIs_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

Today is Monday, tomorrow is
 Tuesday
Today is Tuesday, tomorrow is
 Wednesday
Today is Wednesday, tomorrow is
 Thursday
Today is Thursday, tomorrow is
 Friday
Today is Friday, tomorrow is
6
Today is Saturday, tomorrow is
 Sunday


In [84]:
# assume monday is the first day, aim to get 4 if ends on Wedn
steer_vec = mean_num_sv
multp = 3

for prompt in tomorrowIs_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

Today is Monday, tomorrow is
0
Today is Tuesday, tomorrow is
0
Today is Wednesday, tomorrow is
4
Today is Thursday, tomorrow is
5
Today is Friday, tomorrow is
6
Today is Saturday, tomorrow is
0


In [ ]:
#
prompt = "This month is July, next month is"
steer_vec = mean_num_sv
multp = 1
steer_by_sae_lastTok(prompt, steer_vec, multp)

8


In [ ]:
#
prompt = "four three two"
steer_vec = mean_num_sv
multp = 1
steer_by_sae_lastTok(prompt, steer_vec, multp)

1


In [87]:
words = ['uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve']
SPnumword_samps = []
for i in range(0, 6):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    SPnumword_samps.append(samp)

In [88]:
steer_vec = mean_num_sv
multp = 0

for prompt in SPnumword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

uno dos tres cuatro
 cinco
dos tres cuatro cinco


tres cuatro cinco seis


cuatro cinco seis siete
 
cinco seis siete ocho


seis siete ocho nueve




In [86]:
steer_vec = mean_num_sv
multp = 1

for prompt in SPnumword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

uno dos tres cuatro
5
dos tres cuatro cinco
6
tres cuatro cinco seis
7
cuatro cinco seis siete
8
cinco seis siete ocho
9
seis siete ocho nueve
0


### nonseq prompts

In [59]:
nonseq_prompts = [ "aa a a", "I AM A CAT", "Bob and Mary went to store", "my favorite colour is"]
steer_vec = mean_num_sv
multp = 3

for prompt in nonseq_prompts:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

aa a a
0
I AM A CAT
0
Bob and Mary went to store
0
my favorite colour is
0


In [60]:
nonseq_prompts = [ "aa a a", "I AM A CAT", "Bob and Mary went to store", "my favorite colour is"]
steer_vec = mean_num_sv
multp = 10

for prompt in nonseq_prompts:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

aa a a
0
I AM A CAT
0
Bob and Mary went to store
0
my favorite colour is
0


we score using the correct expected answer

## get sv without subtraction- numerals (no 10)

In [111]:
words = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]}"
    numword_samps.append(samp)
inputs = tokenizer(numword_samps, return_tensors="pt", padding=True, truncation=True, max_length=300)['input_ids'].to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [112]:
# nw_sv = sae_acts_1
mean_num_sv = sae_acts_1.mean(dim=0)
mean_num_sv = mean_num_sv.unsqueeze(0)
mean_num_sv.shape

torch.Size([1, 2, 16384])

### steer

In [113]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    numword_samps.append(samp)

In [114]:
steer_vec = mean_num_sv
multp = 1

for prompt in numword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

one two three four
5
two three four five
6
three four five six
7
four five six seven
8
five six seven eight
9
six seven eight nine
8
seven eight nine ten
1
eight nine ten eleven
2
nine ten eleven twelve
3


In [115]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]}" # {words[i+3]}"
    numword_samps.append(samp)

In [116]:
steer_vec = mean_num_sv
multp = 1

for prompt in numword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

one two three
4
two three four
5
three four five
6
four five six
7
five six seven
8
six seven eight
8
seven eight nine
0
eight nine ten
0
nine ten eleven
2


In [117]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]}" # {words[i+2]} {words[i+3]}"
    numword_samps.append(samp)

In [118]:
steer_vec = mean_num_sv
multp = 1

for prompt in numword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

one two
0
two three
4
three four
0
four five
6
five six
7
six seven
0
seven eight
0
eight nine
0
nine ten
0


In [119]:
words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
months_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    months_samps.append(samp)

In [120]:
steer_vec = mean_num_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

January February March April
5
February March April May
6
March April May June
7
April May June July
8
May June July August
9
June July August September
 October
July August September October
 
August September October November
 
September October November December
 


In [121]:
words = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
months_samps = []
for i in range(0, 5):
    samp = f"{words[i]} {words[i+1]} {words[i+2]}" # {words[i+3]}"
    months_samps.append(samp)

In [122]:
# assume monday is the first day, aim to get 4 if ends on Wedn
steer_vec = mean_num_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
4
Tuesday Wednesday Thursday
5
Wednesday Thursday Friday
 
Thursday Friday Saturday


Friday Saturday Sunday
.


In [123]:
# assume monday is the first day, aim to get 4 if ends on Wedn
steer_vec = mean_num_sv
multp = 3

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
4
Tuesday Wednesday Thursday
5
Wednesday Thursday Friday
0
Thursday Friday Saturday
0
Friday Saturday Sunday
1


In [137]:
words = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
tomorrowIs_samps = []
for i in range(0, 6):
    samp = f"Today is {words[i]}, tomorrow is"
    tomorrowIs_samps.append(samp)

In [138]:
# assume monday is the first day, aim to get 4 if ends on Wedn
steer_vec = mean_num_sv
multp = 1

for prompt in tomorrowIs_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

Today is Monday, tomorrow is
 Tuesday
Today is Tuesday, tomorrow is
 
Today is Wednesday, tomorrow is
 Thursday
Today is Thursday, tomorrow is
 Friday
Today is Friday, tomorrow is
6
Today is Saturday, tomorrow is
 Sunday


In [139]:
words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
nextMonthIs_samps = []
for i in range(0, 6):
    samp = f"This month is {words[i]}, next month is"
    nextMonthIs_samps.append(samp)

In [141]:
#
steer_vec = mean_num_sv
multp = 1

for prompt in nextMonthIs_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

This month is January, next month is
 February
This month is February, next month is
3
This month is March, next month is
 April
This month is April, next month is
5
This month is May, next month is
6
This month is June, next month is
7


In [142]:
#
steer_vec = mean_num_sv
multp = 3

for prompt in nextMonthIs_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

This month is January, next month is
2
This month is February, next month is
3
This month is March, next month is
4
This month is April, next month is
5
This month is May, next month is
6
This month is June, next month is
7


In [146]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve'][::-1]
backw_numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    backw_numword_samps.append(samp)

In [148]:
steer_vec = mean_num_sv
multp = 1

for prompt in backw_numword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

twelve eleven ten nine
8
eleven ten nine eight
7
ten nine eight seven
6
nine eight seven six
0
eight seven six five
 
seven six five four
3
six five four three
2
five four three two
1
four three two one
2


In [149]:
steer_vec = mean_num_sv
multp = 3

for prompt in backw_numword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

twelve eleven ten nine
8
eleven ten nine eight
0
ten nine eight seven
6
nine eight seven six
0
eight seven six five
0
seven six five four
3
six five four three
2
five four three two
1
four three two one
0


In [129]:
words = ['uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve']
SPnumword_samps = []
for i in range(0, 6):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    SPnumword_samps.append(samp)

In [130]:
steer_vec = mean_num_sv
multp = 0

for prompt in SPnumword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

uno dos tres cuatro
 cinco
dos tres cuatro cinco


tres cuatro cinco seis


cuatro cinco seis siete
 
cinco seis siete ocho


seis siete ocho nueve




In [131]:
steer_vec = mean_num_sv
multp = 1

for prompt in SPnumword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

uno dos tres cuatro
5
dos tres cuatro cinco
6
tres cuatro cinco seis
7
cuatro cinco seis siete
8
cinco seis siete ocho
9
seis siete ocho nueve
0


### nonseq prompts

In [132]:
nonseq_prompts = [ "aa a a", "I AM A CAT", "Bob and Mary went to store", "my favorite colour is"]
steer_vec = mean_num_sv
multp = 3

for prompt in nonseq_prompts:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

aa a a
0
I AM A CAT
0
Bob and Mary went to store
0
my favorite colour is
0


In [133]:
nonseq_prompts = [ "aa a a", "I AM A CAT", "Bob and Mary went to store", "my favorite colour is"]
steer_vec = mean_num_sv
multp = 10

for prompt in nonseq_prompts:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

aa a a
0
I AM A CAT
0
Bob and Mary went to store
0
my favorite colour is
0


we score using the correct expected answer

## get nw sv without subtraction

In [71]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']
numword_samps = []
for i in range(0, 10):
    samp = f"{words[i]}"
    numword_samps.append(samp)
inputs = tokenizer(numword_samps, return_tensors="pt", padding=True, truncation=True, max_length=300)['input_ids'].to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [72]:
mean_nw_sv = sae_acts_1.mean(dim=0)
mean_nw_sv = mean_nw_sv.unsqueeze(0)
mean_nw_sv.shape

torch.Size([1, 2, 16384])

### steer

In [78]:
words = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
numerals_samps = []
for i in range(0, 6):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    numerals_samps.append(samp)

In [79]:
steer_vec = mean_nw_sv
multp = 1

for prompt in numerals_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

1 2 3 4
 
2 3 4 5
 
3 4 5 6
 
4 5 6 7
 
5 6 7 8
 
6 7 8 9
 


In [73]:
words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
months_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    months_samps.append(samp)

In [74]:
steer_vec = mean_nw_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

January February March April
 May
February March April May
 
March April May June
 
April May June July


May June July August
 September
June July August September
 October
July August September October
 November
August September October November
 
September October November December




In [75]:
steer_vec = mean_nw_sv
multp = 3

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

January February March April
-
February March April May
-
March April May June
-
April May June July
-
May June July August
-
June July August September
-
July August September October
-
August September October November
-
September October November December
-


## get sv as nw-months

In [ ]:
words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October'] # , 'November', 'December'
month_samps = []
for i in range(0, 10):
    samp = f"{words[i]}"
    month_samps.append(samp)
inputs = tokenizer(month_samps, return_tensors="pt", padding=True, truncation=True, max_length=300)['input_ids'].to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

pair each num prompt with its months analogue, and subtract

In [ ]:
months_minus_nw_nonseq = sae_acts_2 - sae_acts_1
mean_months_minus_nw_nonseq = months_minus_nw_nonseq.mean(dim=0)
mean_months_minus_nw_nonseq = mean_months_minus_nw_nonseq.unsqueeze(0)
mean_months_minus_nw_nonseq.shape

torch.Size([1, 2, 16384])

### steer

In [ ]:
prompt = "four five six"
steer_vec = mean_months_minus_nw_nonseq
multp = 4
steer_by_sae_lastTok(prompt, steer_vec, multp)

In [ ]:
prompt = "four"
steer_vec = mean_months_minus_nw_nonseq
multp = 1
steer_by_sae_lastTok(prompt, steer_vec, multp)

In [ ]:
prompt = "four"
steer_vec = mean_months_minus_nw_nonseq
multp = 5
steer_by_sae_lastTok(prompt, steer_vec, multp)

 måned


In [ ]:
prompt = "one"
steer_vec = mean_months_minus_nw_nonseq
multp = 10
steer_by_sae_lastTok(prompt, steer_vec, multp)

 måned


## mod 10

In [ ]:
#
prompt = "thirteen fourteen fifteen sixteen "
steer_vec = mean_num_sv
multp = 0
steer_by_sae_lastTok(prompt, steer_vec, multp)

1


In [ ]:
#
prompt = "seven eight nine ten eleven"
steer_vec = mean_num_sv
multp = 0
steer_by_sae_lastTok(prompt, steer_vec, multp)

 twelve


In [ ]:
#
prompt = "seven eight nine ten eleven"
steer_vec = mean_num_sv
multp = 1.5
steer_by_sae_lastTok(prompt, steer_vec, multp)

2


In [ ]:
#
prompt = "six seven eight nine ten eleven twelve"
steer_vec = mean_num_sv
multp = 0
steer_by_sae_lastTok(prompt, steer_vec, multp)

# steer by LLM actvs

In [52]:
words = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]}"
    numword_samps.append(samp)
inputs = tokenizer(numword_samps, return_tensors="pt", padding=True, truncation=True, max_length=300)['input_ids'].to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
target_act.shape

torch.Size([9, 2, 2304])

In [48]:
words = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
numword_samps = []
for i in range(0, 10):
    samp = f"{words[i]}"
    numword_samps.append(samp)
inputs = tokenizer(numword_samps, return_tensors="pt", padding=True, truncation=True, max_length=300)['input_ids'].to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
# sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [50]:
target_act.shape

torch.Size([10, 3, 2304])

In [53]:
LLM_mean_num_sv = target_act.mean(dim=0)
LLM_mean_num_sv = LLM_mean_num_sv.unsqueeze(0)
LLM_mean_num_sv.shape

torch.Size([1, 2, 2304])

## steer

In [55]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    numword_samps.append(samp)

In [57]:
steer_vec = LLM_mean_num_sv
multp = 1

for prompt in numword_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

one two three four
5
two three four five
6
three four five six
7
four five six seven
8
five six seven eight
9
six seven eight nine
.
seven eight nine ten
 
eight nine ten eleven
.
nine ten eleven twelve
3


In [66]:
words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
months_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    months_samps.append(samp)

In [68]:
steer_vec = LLM_mean_num_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

January February March April
 May
February March April May
 June
March April May June
 July
April May June July
 August
May June July August
 September
June July August September
 October
July August September October
 November
August September October November
 December
September October November December
 


In [69]:
steer_vec = LLM_mean_num_sv
multp = 3

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

January February March April
5
February March April May
 
March April May June
 
April May June July
 
May June July August
 
June July August September
 
July August September October
 
August September October November
 
September October November December
 


In [70]:
steer_vec = LLM_mean_num_sv
multp = 10

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

January February March April
 
February March April May
 
March April May June
 
April May June July
 
May June July August
 
June July August September
 
July August September October
 
August September October November
 
September October November December
 


In [ ]:
words = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
months_samps = []
for i in range(0, 5):
    samp = f"{words[i]} {words[i+1]} {words[i+2]}" # {words[i+3]}"
    months_samps.append(samp)

In [65]:
#unsteered
steer_vec = LLM_mean_num_sv
multp = 0

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
 Thursday
Tuesday Wednesday Thursday
 Friday
Wednesday Thursday Friday
 Saturday
Thursday Friday Saturday
 Sunday
Friday Saturday Sunday





In [61]:
steer_vec = LLM_mean_num_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
 Thursday
Tuesday Wednesday Thursday
 Friday
Wednesday Thursday Friday
 
Thursday Friday Saturday
 
Friday Saturday Sunday
.


In [63]:
steer_vec = LLM_mean_num_sv
multp = 3

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
4
Tuesday Wednesday Thursday
 
Wednesday Thursday Friday
 
Thursday Friday Saturday
 
Friday Saturday Sunday
 


In [64]:
steer_vec = LLM_mean_num_sv
multp = 10

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
 
Tuesday Wednesday Thursday
 
Wednesday Thursday Friday
 
Thursday Friday Saturday
 
Friday Saturday Sunday
 


# Decompose the LLM steering vec

In [161]:
words = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]}"
    numword_samps.append(samp)
inputs = tokenizer(numword_samps, return_tensors="pt", padding=True, truncation=True, max_length=300)['input_ids'].to("cuda")
target_act = gather_residual_activations(model, 20, inputs)

LLM_mean_num_sv = target_act.mean(dim=0)
LLM_mean_num_sv = LLM_mean_num_sv.unsqueeze(0)
LLM_mean_num_sv.shape

torch.Size([1, 2, 2304])

In [173]:
# decompose LLM steering vec
sae_LLM_mean_num_sv = sae.encode(LLM_mean_num_sv.to(torch.float32))

In [174]:
feat_k = 100
one_top_acts_values, one_top_acts_indices = sae_LLM_mean_num_sv[0, -1, :].topk(feat_k, dim=-1)

In [175]:
rank = 1
for val, ind in zip(one_top_acts_values, one_top_acts_indices):
    print(f"Rank {rank}", round(val.item(), 2), ind.item(), feat_labels_lst[ind])
    rank += 1

Rank 1 150.1 13528 numeric values indicating measurements or quantities
Rank 2 85.52 11527 the start of a document
Rank 3 34.94 9768 terms related to control and authority, particularly in political or systemic contexts
Rank 4 34.91 8684  technical jargon and programming-related terms
Rank 5 33.51 140  instances of the word "in"
Rank 6 30.87 833 the numerical values indicating measurements or assessments
Rank 7 29.39 11174  numeric identifiers or values in a structured format
Rank 8 21.74 2437  patterns related to numerical information, particularly involving the number four
Rank 9 21.66 6305 mathematical expressions and notations used in equations and proofs
Rank 10 15.28 3019  elements related to operational or procedural contexts in a structured format
Rank 11 10.27 11795 the phrase "The" at the start of sentences
Rank 12 9.79 745  sequences of numerical values
Rank 13 9.0 3435  numeric values, particularly related to technology specifications or measurements
Rank 14 8.34 1344 lists

These are nonzero:
- Rank 1 150.1 13528 numeric values indicating measurements or quantities
- Rank 2 85.52 11527 the start of a document
- Rank 3 34.94 9768 terms related to control and authority, particularly in political or systemic contexts
- Rank 4 34.91 8684  technical jargon and programming-related terms
- Rank 5 33.51 140  instances of the word "in"
- Rank 6 30.87 833 the numerical values indicating measurements or assessments
- Rank 7 29.39 11174  numeric identifiers or values in a structured format
- Rank 8 21.74 2437  patterns related to numerical information, particularly involving the number four
- Rank 9 21.66 6305 mathematical expressions and notations used in equations and proofs
- Rank 10 15.28 3019  elements related to operational or procedural contexts in a structured format
- Rank 11 10.27 11795 the phrase "The" at the start of sentences
- Rank 12 9.79 745  sequences of numerical values
- Rank 13 9.0 3435  numeric values, particularly related to technology specifications or measurements
- Rank 14 8.34 1344 lists and resources for educational purposes
- Rank 15 7.86 16175 numeric identifiers or codes, particularly in a structured format or specification

Try ablating the following:
- Rank 2 85.52 11527 the start of a document
- Rank 3 34.94 9768 terms related to control and authority, particularly in political or systemic contexts
- Rank 4 34.91 8684  technical jargon and programming-related terms
- Rank 5 33.51 140  instances of the word "in"
- Rank 10 15.28 3019  elements related to operational or procedural contexts in a structured format
- Rank 11 10.27 11795 the phrase "The" at the start of sentences



## ablate then recon to steer

In [ ]:
def ablate_SAE_lastTok(prompt, steer_vec, multp):
    tokens = model_2.to_tokens(prompt).to(device)
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
    target_act = gather_residual_activations(model, 20, inputs)
    sae_acts_3 = sae.encode(target_act.to(torch.float32))

    sae_acts_3[:, -1, :]  += multp * steer_vec[:, -1, :] # in-place
    recon = sae.decode(sae_acts_3)
    recon.shape

In [178]:
abl_sae_LLM_mean_num_sv = torch.clone(sae_LLM_mean_num_sv)

feats_to_ablate = [11527, 9768, 8684, 140, 3019, 11795]

for fInd in feats_to_ablate:
    abl_sae_LLM_mean_num_sv[:, -1, fInd] = 0

In [179]:
recon_abl_sae_LLM_mean_num_sv = sae.decode(abl_sae_LLM_mean_num_sv)
recon_abl_sae_LLM_mean_num_sv.shape

torch.Size([1, 2, 2304])

## steer ablated recon in LLM space

In [180]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    numword_samps.append(samp)

In [181]:
steer_vec = recon_abl_sae_LLM_mean_num_sv
multp = 1

for prompt in numword_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

one two three four
 
two three four five
 
three four five six
 
four five six seven
 eight
five six seven eight
 nine
six seven eight nine
 
seven eight nine ten
 
eight nine ten eleven
 
nine ten eleven twelve
 


In [184]:
steer_vec = recon_abl_sae_LLM_mean_num_sv
multp = 3

for prompt in numword_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

one two three four
 
two three four five
 
three four five six
 
four five six seven
 
five six seven eight
 
six seven eight nine
 
seven eight nine ten
 
eight nine ten eleven
 
nine ten eleven twelve
 


In [182]:
words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
months_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    months_samps.append(samp)

In [183]:
steer_vec = recon_abl_sae_LLM_mean_num_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

January February March April
 May
February March April May
 June
March April May June
 
April May June July
 August
May June July August
 
June July August September
 October
July August September October
 
August September October November
 December
September October November December
 


In [185]:
steer_vec = recon_abl_sae_LLM_mean_num_sv
multp = 3

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

January February March April
 
February March April May
 
March April May June
 
April May June July
 
May June July August
 
June July August September
 
July August September October
 
August September October November
 
September October November December
 


In [ ]:
steer_vec = LLM_mean_num_sv
multp = 10

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

January February March April
 
February March April May
 
March April May June
 
April May June July
 
May June July August
 
June July August September
 
July August September October
 
August September October November
 
September October November December
 


In [ ]:
words = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
months_samps = []
for i in range(0, 5):
    samp = f"{words[i]} {words[i+1]} {words[i+2]}" # {words[i+3]}"
    months_samps.append(samp)

In [ ]:
#unsteered
steer_vec = LLM_mean_num_sv
multp = 0

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
 Thursday
Tuesday Wednesday Thursday
 Friday
Wednesday Thursday Friday
 Saturday
Thursday Friday Saturday
 Sunday
Friday Saturday Sunday





In [ ]:
steer_vec = LLM_mean_num_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
 Thursday
Tuesday Wednesday Thursday
 Friday
Wednesday Thursday Friday
 
Thursday Friday Saturday
 
Friday Saturday Sunday
.


In [ ]:
steer_vec = LLM_mean_num_sv
multp = 3

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
4
Tuesday Wednesday Thursday
 
Wednesday Thursday Friday
 
Thursday Friday Saturday
 
Friday Saturday Sunday
 


In [ ]:
steer_vec = LLM_mean_num_sv
multp = 10

for prompt in months_samps:
    print(prompt)
    steer_by_LLM_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
 
Tuesday Wednesday Thursday
 
Wednesday Thursday Friday
 
Thursday Friday Saturday
 
Friday Saturday Sunday
 


## steer ablated recon in SAE space

In [186]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    numword_samps.append(samp)

In [189]:
steer_vec = abl_sae_LLM_mean_num_sv
multp = 1

for prompt in numword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

one two three four
5
two three four five
6
three four five six
7
four five six seven
8
five six seven eight
9
six seven eight nine


seven eight nine ten


eight nine ten eleven


nine ten eleven twelve




In [191]:
steer_vec = abl_sae_LLM_mean_num_sv
multp = 3

for prompt in numword_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

one two three four
5
two three four five
6
three four five six
7
four five six seven
8
five six seven eight
9
six seven eight nine
0
seven eight nine ten
1
eight nine ten eleven
1
nine ten eleven twelve
3


In [192]:
words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
months_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    months_samps.append(samp)

In [193]:
steer_vec = abl_sae_LLM_mean_num_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

January February March April
 
February March April May
 
March April May June
 
April May June July
8
May June July August
 
June July August September
 October
July August September October
 
August September October November
 
September October November December
 


In [195]:
steer_vec = abl_sae_LLM_mean_num_sv
multp = 3

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

January February March April
5
February March April May
6
March April May June
7
April May June July
8
May June July August
9
June July August September
0
July August September October
1
August September October November
2
September October November December
1


In [196]:
words = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
months_samps = []
for i in range(0, 5):
    samp = f"{words[i]} {words[i+1]} {words[i+2]}" # {words[i+3]}"
    months_samps.append(samp)

In [197]:
steer_vec = abl_sae_LLM_mean_num_sv
multp = 1

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
 
Tuesday Wednesday Thursday
 
Wednesday Thursday Friday
 
Thursday Friday Saturday
 
Friday Saturday Sunday
,


In [198]:
steer_vec = abl_sae_LLM_mean_num_sv
multp = 3

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
4
Tuesday Wednesday Thursday
5
Wednesday Thursday Friday
0
Thursday Friday Saturday
0
Friday Saturday Sunday
1


In [200]:
steer_vec = abl_sae_LLM_mean_num_sv
multp = 10

for prompt in months_samps:
    print(prompt)
    steer_by_sae_lastTok(prompt, steer_vec, multp)

Monday Tuesday Wednesday
4
Tuesday Wednesday Thursday
5
Wednesday Thursday Friday
0
Thursday Friday Saturday
0
Friday Saturday Sunday
1


# Ablate directly from SAE steering vec

In [168]:
words = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]}"
    numword_samps.append(samp)
inputs = tokenizer(numword_samps, return_tensors="pt", padding=True, truncation=True, max_length=300)['input_ids'].to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [155]:
mean_num_sv = sae_acts_1.mean(dim=0)
mean_num_sv = mean_num_sv.unsqueeze(0)
mean_num_sv.shape

torch.Size([1, 2, 16384])

In [169]:
feat_k = 100 # 50
one_top_acts_values, one_top_acts_indices = mean_num_sv[0, -1, :].topk(feat_k, dim=-1)

In [170]:
rank = 1
for val, ind in zip(one_top_acts_values, one_top_acts_indices):
    print(f"Rank {rank}", round(val.item(), 2), ind.item(), feat_labels_lst[ind])
    rank += 1

Rank 1 150.1 13528 numeric values indicating measurements or quantities
Rank 2 85.52 11527 the start of a document
Rank 3 34.94 9768 terms related to control and authority, particularly in political or systemic contexts
Rank 4 34.91 8684  technical jargon and programming-related terms
Rank 5 33.51 140  instances of the word "in"
Rank 6 30.87 833 the numerical values indicating measurements or assessments
Rank 7 28.97 11174  numeric identifiers or values in a structured format
Rank 8 21.66 6305 mathematical expressions and notations used in equations and proofs
Rank 9 21.37 2437  patterns related to numerical information, particularly involving the number four
Rank 10 15.08 3019  elements related to operational or procedural contexts in a structured format
Rank 11 12.1 3435  numeric values, particularly related to technology specifications or measurements
Rank 12 11.92 745  sequences of numerical values
Rank 13 8.7 3371  numerical expressions and equations
Rank 14 7.24 16175 numeric ide